In [ ]:
from collections import defaultdict
import json
import ee
import pandas as pd
import geopandas as gpd
import geemap

In [ ]:
ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
scale = 300
YEARS = [2010, 2015, 2020]

In [ ]:
with open('popcodes.json', 'r') as popfile:
    popcodes = json.loads(popfile.read())

In [ ]:
bycountry = {}
for year in YEARS:
    print('\n\n{0}\n'.format(year))
    
    pop_img = ee.ImageCollection("JRC/GHSL/P2023A/GHS_POP").filter(ee.Filter.calendarRange(year, year, 'year')).first()
    
    Summer_UHI_yearly_1km=ee.Image('projects/ee-yceo/assets/Summer_UHI_yearly_300_v5')
    Summer_Night_1km = Summer_UHI_yearly_1km.select(['summernighttime_UHI_{0}'.format(year)],['Summernighttime'])
    Summer_Day_1km = Summer_UHI_yearly_1km.select(['summerdaytime_UHI_{0}'.format(year)],['Summerdaytime'])
    
    urbext = ee.FeatureCollection('projects/wri-datalab/SCL-Cities/urbanextents__bycountry_{0}_v7'.format(year))
    urbext_data = geemap.ee_to_pandas(urbext)
    nuhi_res = defaultdict(list)
    duhi_res = defaultdict(list)
    pop_res = defaultdict(list)
    nuhipop_tot = []
    duhipop_tot = []
    pop_tot = []
    for i in range(len(urbext_data)):
        ua = urbext_data.iloc[i]
        if ua['country'] in popcodes:
            ua_f = urbext.filter(ee.Filter.eq('city_ids', str(ua['city_ids'])))
            geom = ua_f.geometry()

            localpop_img = worldpop.filter(ee.Filter.equals('country', ua['country'])).select('population').first()
            localpop = pop_img.reduceRegion(ee.Reducer.sum(), geom, scale, maxPixels=1e12).getInfo()['population_count']

            localpop_nuhi = localpop_img.multiply(Summer_Night_1km).reduceRegion(ee.Reducer.sum(), geom, scale, maxPixels=1e12).getInfo()['population']
            localpop_duhi = localpop_img.multiply(Summer_Day_1km).reduceRegion(ee.Reducer.sum(), geom, scale, maxPixels=1e12).getInfo()['population']

            nuhi_res[ua['country']].append(localpop_nuhi)
            duhi_res[ua['country']].append(localpop_duhi)
            pop_res[ua['country']].append(localpop)

            nuhipop_tot.append(localpop_nuhi)
            duhipop_tot.append(localpop_duhi)
            pop_tot.append(localpop)

            print(i, end=' ')
    if not 'global' in list(bycountry.keys()):
        bycountry['global'] = {}
    bycountry['global'] = bycountry['global'] | {
        'pop-averaged_summer_nighttime_uhi_{0}'.format(year): sum(nuhipop_tot) / sum(pop_tot),
        'pop-averaged_summer_daytime_uhi_{0}'.format(year): sum(duhipop_tot) / sum(pop_tot),
    }
    country_list = list(nuhi_res.keys())
    country_list.sort()
    for country in country_list:
        if not country in list(bycountry.keys()):
            bycountry[country] = {}
        if sum(nuhi_res[country]):
            bycountry[country] = bycountry[country] | {
                'pop-averaged_summer_nighttime_uhi_{0}'.format(year): sum(nuhi_res[country]) / sum(pop_res[country]),
                'pop-averaged_summer_daytime_uhi_{0}'.format(year): sum(duhi_res[country]) / sum(pop_res[country]),
            }
        else:
            bycountry[country] = bycountry[country] | {
                'pop-averaged_summer_nighttime_uhi_{0}'.format(year): -9999,
                'pop-averaged_summer_daytime_uhi_{0}'.format(year): -9999,
            }
res_pd = pd.DataFrame.from_dict(bycountry, orient='index')
res_pd.to_csv('CTY-11_uhi_v7.csv')

In [ ]:
out_dicts = []

out_dicts.append(
        {
            'country': 'global',
            'pop-averaged_summer_nighttime_uhi': sum(nuhipop_tot) / sum(pop_tot),
            'pop-averaged_summer_daytime_uhi': sum(duhipop_tot) / sum(pop_tot),
        }
    )

country_list = list(nuhi_res.keys())
country_list.sort()
for country in country_list:
    if sum(nuhi_res[country]):
        out_dicts.append(
            {
                'country': country,
                'pop-averaged_summer_nighttime_uhi': sum(nuhi_res[country]) / sum(pop_res[country]),
                'pop-averaged_summer_daytime_uhi': sum(duhi_res[country]) / sum(pop_res[country]),
            }
        )
    else:
        out_dicts.append(
            {
                'country': country,
                'pop-averaged_summer_nighttime_uhi': -9999,
                'pop-averaged_summer_daytime_uhi': -9999,
            }
        )
res_pd = pd.DataFrame(out_dicts)
res_pd.to_csv('CTY-11_uhi.csv')